# OpenAltimetry ICESat-2: Photon ATL03


### Overview
<blockquote>This notebook demonstrates how to pull low level global geolocated photon data (ATL03) from the ICESat-2 mission via the OpenAltimetry API (https://openaltimetry.org/data/swagger-ui/).</blockquote>



### Instructions
<blockquote>Replace the "OA_API_URL" string with the data copied from : "Get API request" from the OpenAltimetry elevation chart and start visualizing your data!</blockquote>

Author: Luis Lopez - luis.lopez@nsidc.org; updated: Minh Phan - mnphan@ucsd.edu

In [1]:
import pandas as pd
import json
import requests
import plotly.graph_objects as go
from plotly.offline import iplot

# Paste the OpenAltimetry API URL for Photon here: 
OA_API_URL = 'https://openaltimetry.org/data/api/icesat2/atl03?&minx=47.45681762695317&miny=33.116912841796854&maxx=47.59826660156254&maxy=33.229522705078104&date=2019-09-06&trackId=1081&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l'

# Select the list of confidence to display: 'Noise', 'Buffer', 'Low', 'Medium', 'High'
OA_PHOTON_CONFIDENCE = ['Noise', 'Buffer', 'Low', 'Medium', 'High']

df_beams = []
oa_plots = []

# This function will request the 6 beams data using OpenAltimetry's API
def getPhotonData():
    series = []
    b_url = OA_API_URL + '&client=jupyter'
    print('Requesting data from OA')
    r = requests.get(b_url)
    data = r.json()
    return data

        
photon_cloud = getPhotonData()

for beam in photon_cloud:
    
    print('Reading data for beam: ' + beam['beam_name'])

    for photons in beam['series']:
        if any(word in photons['name'] for word in OA_PHOTON_CONFIDENCE):
            series = []
            for p in photons['data']:
                series.append({
                    'lat': p[0],
                    'lon': p[1],
                    'h': p[2]
                })
            if (len(series) > 0):
                df = pd.DataFrame.from_dict(series)
                df.name = beam['beam_name'] + ' ' + photons['name']
                df_beams.append(df)

for df in df_beams:
    oa_plots.append(go.Scatter3d(name=df.name, x=df['lat'], y=df['lon'], z=df['h'], mode='markers',
                                    marker=dict(
                                        size=1,
                                        color=df['h'],
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
                                    )
                                )
                   )
    
layout = go.Layout(
    width=900,
    height=600,
    scene = dict(aspectmode = 'manual', aspectratio =dict(x=1, y=1, z=0.5),
                 xaxis=dict(title='Latitude'), yaxis=dict(title='Longitude'), zaxis=dict(title='Elevation (m)'))
)

print('Plotting...')

fig = go.Figure(data=oa_plots, layout=layout)  

iplot(fig)


In [ ]:
# Now we can get a histogram of the beam's elevation and their description
for df in df_beams:
    print(df.describe())
    df.hist('h', bins=128)